##Import and install required libraries

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Conv2D,Dense,MaxPooling2D,Dropout,BatchNormalization,Activation 
from PIL import Image
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 181 kB 46.8 MB/s 
     |████████████████████████████████| 144 kB 45.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e2014b429be7ce55f2543d89322caa7c82f9b35ca2cc12cd12d8e30afaef160e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Unzip and load file data file onto server, then delete zip file

In [ ]:
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

## Prepare the dataset for Training

In [ ]:
def dataset_Training(Dataset_loc="inaturalist_12K", augment_data=False,ts=(299,299)):
    Directory_train= os.path.join(Dataset_loc, "train")
    Directory_test= os.path.join(Dataset_loc, "val")

    if augment_data:
        generator_train=ImageDataGenerator(rescale=1./255,rotation_range=90,zoom_range=0.2,shear_range=0.2,horizontal_flip=True)
    else:
        generator_train=ImageDataGenerator(rescale=1./255)
       
    generator_test = ImageDataGenerator(rescale=1./255)
    train_generate = generator_train.flow_from_directory(Directory_train, target_size=ts, batch_size=128, subset="training",shuffle=True,seed=8700)
    #val_generate = generator_train.flow_from_directory(Directory_train, target_size=ts, batch_size=128, subset="validation",shuffle=True,seed=8700)
    test_generate = generator_test.flow_from_directory(Directory_test, target_size=ts, batch_size=128)
    
    return train_generate, test_generate
train_data, val_data = dataset_Training()

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


##Our Model InceptionResNetV2,InceptionV3,ResNet50, Xception

In [ ]:
def our_Model(Model_name):
  
  if Model_name=="InceptionResNetV2":
    w,ht=299,299
    return tf.keras.applications.InceptionResNetV2(include_top=False, input_shape=(w,ht,3), weights='imagenet')
  elif Model_name=="InceptionV3":
    w,ht=299,299
    return tf.keras.applications.InceptionV3(include_top=False, input_shape=(w,ht,3), weights='imagenet')
  elif Model_name=="Xception":
    w,ht=299,299
    return tf.keras.applications.Xception(include_top=False, input_shape=(w,ht,3), weights='imagenet')
  elif Model_name=="ResNet50":
    w,ht=224,224
    return tf.keras.applications.ResNet50(include_top=False, input_shape=(w,ht,3), weights='imagenet')
 

##Fine Tuning the model 

In [ ]:
def train_model():
  config_defaults = {
        "base_model": "InceptionV3",
        "lr": 0.001,
        "batch_normalisation":False,
        "epochs": 10,
        "augment_data":False,
        "dropout":0.4,
        "dense_Size":128
        }
  wandb.init(config=config_defaults, project="cs6910_assignment212", magic=True)
  config = wandb.config
  ## 1. Data loading
  img_height,img_width=(299,299)
  print("1. Loading the dataset ...\n")
  if config.base_model=="ResNet50":
    img_height,img_width=224,224
    train_data, val_data=dataset_Training(augment_data=config.augment_data,ts=(img_height,img_width))
  else:
     train_data, val_data=dataset_Training(augment_data=config.augment_data,ts=(img_height,img_width))
  O_model=our_Model(config.base_model)

  for layers in O_model.layers:
    layers.trainable = False

  model = keras.Sequential([
    tf.keras.Input(shape=(img_height, img_width,3,)),
    O_model,
    Flatten(),
    Dense(config.dense_Size,activation='relu'),
      
  ])
  if config.batch_normalisation == True:
    model.add(BatchNormalization())
  model.add(Dropout(config.dropout))
  model.add(Dense(config.dense_Size,activation='relu'))
  model.add(Dropout(config.dropout))
  model.add(Dense(10 ,activation='softmax'))

  #Without Fine tuning the model .
  # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.lr),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])  
  # model.fit(train_data, validation_data=val_data,epochs=config.epochs,callbacks=[WandbCallback()])
  
  tune_defaults = {
        "InceptionV3": 55,
        "InceptionResNetV2": 55,
        "ResNet50": 50,
        "Xception": 50
  } 
  model.trainable =True
  print(f"Total layers in base model is {len(model.layers)}\n")

  fine_tune= len(model.layers) - int(.01*len(model.layers)*tune_defaults[config.base_model])
  for layer in model.layers[:fine_tune]:
      layer.trainable =False

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.lr),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])  
  print("Fine tuning the model ...\n")
  model.fit(train_data, validation_data=val_data,epochs=config.epochs,callbacks=[WandbCallback()])
  print("Model tuned successfully!!\n")

##All sweep

In [ ]:

sweep_config_final = {
  "name": "My Sweep",
  "method": "random",
  "parameters": {
        "base_model": {
            "values": ["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception"]
        },
         "augment_data": {
            "values": [True,False]
        },
         "batch_normalisation": {
            "values": [True,False]
        },
        "lr": {
            "values": [0.00007,0.0002]
        },
        "epochs": {
            "values": [5,10]
        },
        "dropout":{
            "values":[0.2,0.37]
        },
        "dense_Size":{
            "values":[128,512]
        }
    }
}


sweep_id = wandb.sweep(sweep_config_final, project="CS6910_Assignment2B")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 7mhcjb80
Sweep URL: https://wandb.ai/manishiitm/CS6910_Assignment2B/sweeps/7mhcjb80


In [ ]:
# running the sweep
wandb.agent(sweep_id, function=train_model,count=40)

wandb: Agent Starting Run: no1i48er with config:
wandb: 	augment_data: True
wandb: 	base_model: ResNet50
wandb: 	batch_normalisation: True
wandb: 	dense_Size: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	lr: 0.0002


#############


wandb: Currently logged in as: manishiitm (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


#############
1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
94781440/94765736 [==============================] - 0s 0us/step
Total layers in base model is 8

yehi h4
Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 269s 3s/step - loss: 2.4020 - accuracy: 0.1073 - val_loss: 2.2753 - val_accuracy: 0.1290 - _timestamp: 1649074734.0000 - _runtime: 281.0000
Epoch 2/10
79/79 [==============================] - 236s 3s/step - loss: 2.3372 - accuracy: 0.1196 - val_loss: 2.2632 - val_accuracy: 0.1605 - _timestamp: 1649074971.0000 - _runtime: 518.0000
Epoch 3/10
79/79 [==============================] - 228s 3s/step - loss: 2.3035 - accuracy: 0.1234 - val_loss: 2.2560 - val_accuracy: 0.1560 - _timestamp: 1649075199.0000 - _runtime: 746.0000
Epoch 4/10
79/79 [==============================] - 230s 3s/step - loss: 2.2854 - accuracy: 0.1333 - val_loss: 2.2569 - val_accuracy: 0.1455 - _timestamp: 16490754

...

accuracy,▁▂▃▄▅▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▆▅▄▃▂▂▁▁
val_accuracy,▁▃▄▅▆▇▇▇██
val_loss,█▇▆▅▅▄▃▂▂▁
accuracy,0.35074
best_epoch,9
best_val_loss,1.66589
epoch,9
loss,1.91249
val_accuracy,0.572


wandb: Agent Starting Run: 4kh1y202 with config:
wandb: 	augment_data: True
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: True
wandb: 	dense_Size: 128
wandb: 	dropout: 0.37
wandb: 	epochs: 5
wandb: 	lr: 7e-05


#############


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


#############
1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Total layers in base model is 8

yehi h4
Fine tuning the model ...

Epoch 1/5
79/79 [==============================] - 305s 4s/step - loss: 2.8298 - accuracy: 0.1096 - val_loss: 2.3656 - val_accuracy: 0.1290 - _timestamp: 1649097527.0000 - _runtime: 319.0000
Epoch 2/5
79/79 [==============================] - 299s 4s/step - loss: 2.6731 - accuracy: 0.1101 - val_loss: 2.3103 - val_accuracy: 0.1405 - _timestamp: 1649097827.0000 - _runtime: 619.0000
Epoch 3/5
79/79 [==============================] - 298s 4s/step - loss: 2.5675 - accuracy: 0.1240 - val_loss: 2.2758 - val_accuracy: 0.1525 - _timestamp: 1649098125.0000 - _runtime: 917.0000
Epoch 4/5
79/79 [==============================] - 299s 4s/step - loss: 2.5277 - accuracy: 0.1212 - val_loss: 2.2481 - val_accuracy: 0.1680 - _timestamp: 1649098423.0000 - _runtime: 1215.0000
Epoch 5/5
44/79 [===============>......

In [ ]:
# running the sweep
wandb.agent(sweep_id, function=train_model,count=40)

wandb: Agent Starting Run: rt67m1yx with config:
wandb: 	augment_data: False
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: True
wandb: 	dense_Size: 512
wandb: 	dropout: 0.37
wandb: 	epochs: 5
wandb: 	lr: 0.0002
wandb: Currently logged in as: manishiitm (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87924736/87910968 [==============================] - 1s 0us/step
Total layers in base model is 8

Fine tuning the model ...

Epoch 1/5
79/79 [==============================] - 272s 3s/step - loss: 2.2647 - accuracy: 0.2238 - val_loss: 1.6519 - val_accuracy: 0.5580 - _timestamp: 1649099315.0000 - _runtime: 286.0000
Epoch 2/5
79/79 [==============================] - 194s 2s/step - loss: 1.7424 - accuracy: 0.4322 - val_loss: 1.2781 - val_accuracy: 0.6745 - _timestamp: 1649099510.0000 - _runtime: 481.0000
Epoch 3/5
79/79 [==============================] - 186s 2s/step - loss: 1.4913 - accuracy: 0.5279 - val_loss: 1.0828 - val_accuracy: 0.7045 - _timestamp: 1649099695.0000 - _runtime: 666.0000
Epoch 4/5
79/79 [==============================] - 187s 2s/step - loss: 1.3415 - accuracy: 0.5787 - val_loss: 0.9795 - val_accuracy: 0.7230 - _timestamp: 1649099899.0000 - _runtime: 870.00

accuracy,▁▅▇██
epoch,▁▃▅▆█
loss,█▄▃▁▁
val_accuracy,▁▆▇██
val_loss,█▄▂▁▁
accuracy,0.60006
best_epoch,4
best_val_loss,0.93359
epoch,4
loss,1.27305
val_accuracy,0.728


wandb: Agent Starting Run: q1efbqi2 with config:
wandb: 	augment_data: True
wandb: 	base_model: InceptionResNetV2
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 512
wandb: 	dropout: 0.37
wandb: 	epochs: 10
wandb: 	lr: 0.0002
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Total layers in base model is 8

Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 185s 2s/step - loss: 2.5101 - accuracy: 0.1128 - val_loss: 2.3350 - val_accuracy: 0.1355 - _timestamp: 1649105624.0000 - _runtime: 196.0000
Epoch 2/10
79/79 [==============================] - 176s 2s/step - loss: 2.4205 - accuracy: 0.1204 - val_loss: 2.2708 - val_accuracy: 0.1645 - _timestamp: 1649105823.0000 - _runtime: 395.0000
Epoch 3/10
79/79 [==============================] - 173s 2s/step - loss: 2.3404 - accuracy: 0.1458 - val_loss: 2.2159 - val_accuracy: 0.2005 - _timestamp: 1649105996.0000 - _runtime: 568.0000
Epoch 4/10
79/79 [==============================] - 173s 2s/step - loss: 2.2866 - accuracy: 0.1660 - val_loss: 2.1658 - val_accuracy: 0.2475 - _timestamp: 1649106198.0000 - _runtime: 770.0000
Epoch 5/10
79/79 [==============================] - 173s 2

accuracy,▁▁▂▃▄▅▆▇▇█
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇▅▅▄▃▃▂▁▁
val_accuracy,▁▂▃▄▄▅▆▇██
val_loss,█▇▆▅▄▄▃▂▂▁
accuracy,0.30403
best_epoch,9
best_val_loss,1.91485
epoch,9
loss,2.04785
val_accuracy,0.419


wandb: Agent Starting Run: v1p449d2 with config:
wandb: 	augment_data: False
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: True
wandb: 	dense_Size: 128
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	lr: 7e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Total layers in base model is 8

Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 183s 2s/step - loss: 2.5158 - accuracy: 0.1046 - val_loss: 2.3635 - val_accuracy: 0.1100 - _timestamp: 1649107496.0000 - _runtime: 193.0000
Epoch 2/10
79/79 [==============================] - 174s 2s/step - loss: 2.4193 - accuracy: 0.1196 - val_loss: 2.2977 - val_accuracy: 0.1405 - _timestamp: 1649107671.0000 - _runtime: 368.0000
Epoch 3/10
79/79 [==============================] - 176s 2s/step - loss: 2.3572 - accuracy: 0.1391 - val_loss: 2.2421 - val_accuracy: 0.1750 - _timestamp: 1649107847.0000 - _runtime: 544.0000
Epoch 4/10
79/79 [==============================] - 176s 2s/step - loss: 2.3020 - accuracy: 0.1579 - val_loss: 2.1920 - val_accuracy: 0.2090 - _timestamp: 1649108023.0000 - _runtime: 720.0000
Epoch 5/10
79/79 [==============================] - ETA: 0

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
# running the sweep
wandb.agent(sweep_id, function=train_model,count=40)

wandb: Agent Starting Run: 1ilje104 with config:
wandb: 	augment_data: True
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 512
wandb: 	dropout: 0.37
wandb: 	epochs: 10
wandb: 	lr: 7e-05
wandb: Currently logged in as: manishiitm (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87924736/87910968 [==============================] - 1s 0us/step
Total layers in base model is 7

Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 418s 5s/step - loss: 2.5751 - accuracy: 0.1258 - val_loss: 2.1090 - val_accuracy: 0.2725 - _timestamp: 1649134239.0000 - _runtime: 436.0000
Epoch 2/10
79/79 [==============================] - 375s 5s/step - loss: 2.3185 - accuracy: 0.2020 - val_loss: 1.9324 - val_accuracy: 0.4230 - _timestamp: 1649134645.0000 - _runtime: 842.0000
Epoch 3/10
79/79 [==============================] - 363s 5s/step - loss: 2.1483 - accuracy: 0.2550 - val_loss: 1.7906 - val_accuracy: 0.5210 - _timestamp: 1649135006.0000 - _runtime: 1203.0000
Epoch 4/10
79/79 [==============================] - 360s 5s/step - loss: 2.0183 - accuracy: 0.3173 - val_loss: 1.6749 - val_accuracy: 0.5595 - _timestamp: 1649135368.0000 - _runtime: 1

accuracy,▁▃▄▅▆▆▇▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▄▅▆▇▇▇███
val_loss,█▇▅▅▄▃▂▂▁▁
accuracy,0.47675
best_epoch,9
best_val_loss,1.22132
epoch,9
loss,1.64195
val_accuracy,0.681


wandb: Agent Starting Run: 4x4s9kvp with config:
wandb: 	augment_data: True
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 128
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	lr: 0.0002
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: yvlt54c8 with config:
wandb: 	augment_data: False
wandb: 	base_model: ResNet50
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	lr: 0.0002
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
94781440/94765736 [==============================] - 1s 0us/step
Total layers in base model is 7

Fine tuning the model ...

Epoch 1/5
79/79 [==============================] - 186s 2s/step - loss: 2.4171 - accuracy: 0.1039 - val_loss: 2.2776 - val_accuracy: 0.1215 - _timestamp: 1649143828.0000 - _runtime: 198.0000
Epoch 2/5
79/79 [==============================] - 173s 2s/step - loss: 2.3275 - accuracy: 0.1230 - val_loss: 2.2562 - val_accuracy: 0.1565 - _timestamp: 1649144001.0000 - _runtime: 371.0000
Epoch 3/5
79/79 [==============================] - 172s 2s/step - loss: 2.2977 - accuracy: 0.1307 - val_loss: 2.2425 - val_accuracy: 0.1690 - _timestamp: 1649144173.0000 - _runtime: 543.0000
Epoch 4/5
79/79 [==============================] - 167s 2s/step - loss: 2.2775 - accuracy: 0.1428 - val_loss: 2.2364 - val_accuracy: 0.1840 - _timestamp: 1649144340.0000 - _runtime: 710.00

accuracy,▁▄▅▇█
epoch,▁▃▅▆█
loss,█▄▂▁▁
val_accuracy,▁▅▆██
val_loss,█▅▂▁▁
accuracy,0.14951
best_epoch,4
best_val_loss,2.23437
epoch,4
loss,2.26694
val_accuracy,0.18


wandb: Agent Starting Run: mc2d3ny4 with config:
wandb: 	augment_data: False
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 512
wandb: 	dropout: 0.37
wandb: 	epochs: 10
wandb: 	lr: 7e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Total layers in base model is 7

Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 200s 2s/step - loss: 2.5608 - accuracy: 0.1458 - val_loss: 2.0693 - val_accuracy: 0.2965 - _timestamp: 1649144741.0000 - _runtime: 210.0000
Epoch 2/10
79/79 [==============================] - 195s 2s/step - loss: 2.2184 - accuracy: 0.2301 - val_loss: 1.8237 - val_accuracy: 0.4790 - _timestamp: 1649144945.0000 - _runtime: 414.0000
Epoch 3/10
57/79 [====================>.........] - ETA: 45s - loss: 2.0160 - accuracy: 0.3134

In [ ]:
# running the sweep
wandb.agent(sweep_id, function=train_model,count=40)

wandb: Agent Starting Run: apew36sh with config:
wandb: 	augment_data: True
wandb: 	base_model: InceptionV3
wandb: 	batch_normalisation: True
wandb: 	dense_Size: 128
wandb: 	dropout: 0.37
wandb: 	epochs: 10
wandb: 	lr: 0.0002
wandb: Currently logged in as: manishiitm (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
87924736/87910968 [==============================] - 1s 0us/step
Total layers in base model is 8

Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 395s 5s/step - loss: 2.6407 - accuracy: 0.1097 - val_loss: 2.2623 - val_accuracy: 0.1530 - _timestamp: 1649338729.0000 - _runtime: 416.0000
Epoch 2/10
79/79 [==============================] - 363s 5s/step - loss: 2.4539 - accuracy: 0.1287 - val_loss: 2.1958 - val_accuracy: 0.2205 - _timestamp: 1649339092.0000 - _runtime: 779.0000
Epoch 3/10
79/79 [==============================] - 358s 5s/step - loss: 2.3389 - accuracy: 0.1502 - val_loss: 2.1465 - val_accuracy: 0.2835 - _timestamp: 1649339468.0000 - _runtime: 1155.0000
Epoch 4/10
79/79 [==============================] - 357s 5s/step - loss: 2.2793 - accuracy: 0.1785 - val_loss: 2.1070 - val_accuracy: 0.3285 - _timestamp: 1649339825.0000 - _runtime: 1

accuracy,▁▂▃▄▅▅▆▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▄▃▃▂▂▂▁▁
val_accuracy,▁▂▄▅▆▆▇▇██
val_loss,█▇▆▅▄▄▃▂▂▁
accuracy,0.27183
best_epoch,9
best_val_loss,1.90276
epoch,9
loss,2.08589
val_accuracy,0.4705


wandb: Agent Starting Run: azrvw1w8 with config:
wandb: 	augment_data: False
wandb: 	base_model: InceptionResNetV2
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 512
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	lr: 0.0002
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

wandb: Agent Starting Run: rgcu858k with config:
wandb: 	augment_data: False
wandb: 	base_model: ResNet50
wandb: 	batch_normalisation: False
wandb: 	dense_Size: 512
wandb: 	dropout: 0.37
wandb: 	epochs: 5
wandb: 	lr: 7e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
94781440/94765736 [==============================] - 3s 0us/step
Total layers in base model is 7

Fine tuning the model ...

Epoch 1/5
79/79 [==============================] - 181s 2s/step - loss: 2.6782 - accuracy: 0.1023 - val_loss: 2.3002 - val_accuracy: 0.1175 - _timestamp: 1649344707.0000 - _runtime: 197.0000
Epoch 2/5
79/79 [==============================] - 168s 2s/step - loss: 2.5457 - accuracy: 0.1070 - val_loss: 2.2937 - val_accuracy: 0.1260 - _timestamp: 1649344875.0000 - _runtime: 365.0000
Epoch 3/5
79/79 [==============================] - 166s 2s/step - loss: 2.4708 - accuracy: 0.1038 - val_loss: 2.2884 - val_accuracy: 0.1595 - _timestamp: 1649345041.0000 - _runtime: 531.0000
Epoch 4/5
79/79 [==============================] - 167s 2s/step - loss: 2.4145 - accuracy: 0.1083 - val_loss: 2.2860 - val_accuracy: 0.1385 - _timestamp: 1649345244.0000 - _runtime: 734.00

accuracy,▁▄▂▅█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▂█▅▆
val_loss,█▅▃▂▁
accuracy,0.11401
best_epoch,4
best_val_loss,2.28425
epoch,4
loss,2.37339
val_accuracy,0.148


wandb: Agent Starting Run: 49oprrrg with config:
wandb: 	augment_data: False
wandb: 	base_model: InceptionResNetV2
wandb: 	batch_normalisation: True
wandb: 	dense_Size: 512
wandb: 	dropout: 0.37
wandb: 	epochs: 10
wandb: 	lr: 0.0002
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Total layers in base model is 8

Fine tuning the model ...

Epoch 1/10
79/79 [==============================] - 280s 3s/step - loss: 2.0981 - accuracy: 0.2889 - val_loss: 1.4466 - val_accuracy: 0.6730 - _timestamp: 1649345731.0000 - _runtime: 297.0000
Epoch 2/10
69/79 [=========================>....] - ETA: 27s - loss: 1.5345 - accuracy: 0.5332